In [ ]:
import os
import pandas as pd

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [ ]:
import bnlearn as bn

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Arial Unicode Ms']
# plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']

In [ ]:
from utils import read_data
combined_data = read_data()

In [221]:
select_group = [
    # 號誌
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',

    # 車道劃分
    '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',

    # 大類別
    '事故類型及型態子類別名稱', '車道劃分設施-分向設施子類別名稱',
    '道路型態子類別名稱',

    # 其他
    '速限-第1當事者',
    '道路類別-第1當事者-名稱',

    # 設施
    'youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count',
    
    # 駕駛、行人行為
    '肇因研判子類別名稱-主要'
    ]

In [222]:
data = combined_data[select_group].copy()
data['facility'] = data[['youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count']].apply(
    lambda row: '1' if (row > 0).any() else '0', axis=1
)
data.drop(columns=['youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count'], inplace=True)

max_speed = data['速限-第1當事者'].max()
bins = range(0, int(max_speed) + 11, 10)

data['速限-第1當事者'] = pd.cut(
    data['速限-第1當事者'],
    bins=bins,
    right=False, 
    include_lowest=True,
    labels=[f"{i}-{i+9}" for i in bins[:-1]]
)

In [223]:
for i in data.columns:
    print(i, len(data[i].unique()))

號誌-號誌種類名稱 4
號誌-號誌動作名稱 4
車道劃分設施-分道設施-快車道或一般車道間名稱 5
車道劃分設施-分道設施-快慢車道間名稱 5
車道劃分設施-分道設施-路面邊線名稱 2
事故類型及型態子類別名稱 30
車道劃分設施-分向設施子類別名稱 6
道路型態子類別名稱 16
速限-第1當事者 12
道路類別-第1當事者-名稱 9
肇因研判子類別名稱-主要 113
facility 2


In [225]:
parent = [
    '號誌-號誌種類名稱', '號誌-號誌動作名稱','車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱',
    '車道劃分設施-分道設施-路面邊線名稱', '車道劃分設施-分向設施子類別名稱', '道路型態子類別名稱',
    '速限-第1當事者', '道路類別-第1當事者-名稱', 'facility'
    ]

cause = ['肇因研判子類別名稱-主要']

result = ['事故類型及型態子類別名稱']

In [226]:
black_list = []
# cause -> parent
black_list += [(c, p) for c in cause for p in parent]
# result -> parent/cause
black_list += [(r, x) for r in result for x in (parent + cause + result)]
# 保險：parent -> result（避免直接 shortcut，如果只想透過肇因解釋）
# black_list += [(p, r) for p in parent for r in result]

In [227]:
# 學哪些變數之間有邊，結果是一個DAG
model = bn.structure_learning.fit(data, methodtype='hc', scoretype='bic', bw_list_method='edges',
                                  # 肇因對於事故類型一定是上游。ex. 不會因為撞路樹而造成患病，而是因為患病才造成撞路樹
                                   black_list=black_list)
# 計算每個節點的 條件機率表 (CPT, Conditional Probability Table)
model_param = bn.parameter_learning.fit(model, data, scoretype='bdeu', methodtype='bayes')
# 計算邊緣強度，如果p小於顯著就是有相關
model_independence = bn.independence_test(model_param, data, test='chi_square', prune=True)

[bnlearn] >Warning: Computing DAG with 12 nodes can take a very long time!
[bnlearn] >Computing best DAG using [hc]
[bnlearn] >Set scoring type at [bic]
[bnlearn] >Filter edges based on black_list/white_list
[bnlearn] >Compute structure scores for model comparison (higher is better).
[bnlearn] >Parameter learning> Computing parameters using [bayes]
[bnlearn] >Converting [<class 'pgmpy.base.DAG.DAG'>] to BayesianNetwork model.
[bnlearn] >Converting adjmat to BayesianNetwork.
[bnlearn] >CPD of 號誌-號誌種類名稱:
+-----------------------------+-----+------------------------+
| 號誌-號誌動作名稱                   | ... | 號誌-號誌動作名稱(無號誌)         |
+-----------------------------+-----+------------------------+
| 號誌-號誌種類名稱(無號誌)              | ... | 0.9990042432514246     |
+-----------------------------+-----+------------------------+
| 號誌-號誌種類名稱(行車管制號誌)           | ... | 0.00033191891619180136 |
+-----------------------------+-----+------------------------+
| 號誌-號誌種類名稱(行車管制號誌(附設行人專用號誌)) | ... | 0.00033191891

In [ ]:
model_independence['independence_test']

In [ ]:
bn.get_parents(model['model_edges'])

In [ ]:
from matplotlib import rcParams
rcParams['font.sans-serif'] = ['Microsoft JhengHei']
rcParams['axes.unicode_minus'] = False

# G = bn.plot(model, interactive=False, node_color="#36AA5B", edge_labels=None)
bn.plot(model_independence, interactive=False, edge_labels='pvalue', 
        params_static={'layout': 'spring_layout', 'font_color': "#974848", 'edge_color': "#974848"})

In [ ]:
import numpy as np
import networkx as nx
import plotly.graph_objects as go
from utils_behaviour import feature_name_map

def draw_bn_plotly(model, layout_algo="", en=False, width=1000, height=500):
    edges = [(str(u), str(v)) for u, v in model['model_edges']]
    df = model['independence_test'][['source','target','p_value']].copy()

    if en:
        df['source'] = df['source'].map(feature_name_map).fillna(df['source'])
        df['target'] = df['target'].map(feature_name_map).fillna(df['target'])
        edges = [(feature_name_map.get(u, u), feature_name_map.get(v, v)) for (u, v) in edges]
    else:
        df['source'] = df['source'].astype(str)
        df['target'] = df['target'].astype(str)

    p_map = {(s,t):p for s,t,p in df.itertuples(index=False, name=None)}
    p_map.update({(t,s):p for (s,t),p in list(p_map.items())})

    G = nx.DiGraph()
    G.add_edges_from(edges)

    pos = (nx.spring_layout(G, seed=42) if layout_algo=="spring"
           else nx.kamada_kawai_layout(G))

    # nodes
    deg = dict(G.degree())
    node_x, node_y, node_text, node_size = [], [], [], []
    for n in G.nodes():
        x,y = pos[n]
        node_x.append(x); node_y.append(y)
        node_text.append(f"{n}<br>degree: {deg.get(n,0)}")
        # node_size.append(10 + 25*(deg.get(n,1)))
        node_size.append(50)

    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers+text',
        text=[str(n) for n in G.nodes()],
        textposition="middle center",
        hovertext=node_text, hoverinfo="text",
        marker=dict(size=node_size, 
                    # line=dict(width=1), 
                    line=dict(color="#24475E", width=2),
                    color="#5390B9")
    )

    # edge
    edge_traces = []
    annotations = []
    r = 0.07

    for u, v in G.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        dx, dy = x1 - x0, y1 - y0
        d = (dx**2 + dy**2)**0.5
        if d == 0:
            continue

        # 起點：從 source 往 target 方向移動 r
        sx = x0 + dx/d * r
        sy = y0 + dy/d * r
        # 終點：從 target 往 source 方向退 r
        ex = x1 - dx/d * r
        ey = y1 - dy/d * r

        annotations.append(dict(
            ax=sx, ay=sy, x=ex, y=ey,
            xref="x", yref="y", axref="x", ayref="y",
            showarrow=True, arrowhead=3, arrowsize=2, opacity=0.8
        ))


    fig = go.Figure(data=edge_traces + [node_trace],
        layout=go.Layout(
            template=None, showlegend=False,
            hovermode='closest',
            margin=dict(l=10, r=10, t=10, b=10),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            annotations=annotations,
            width=width, height=height,
        )
    )
    return fig

draw_bn_plotly(model_independence, layout_algo='', en=False, width=900, height=700)


In [ ]:
# Conditional Probability Distributions (CPDs)
CPDs = bn.print_CPD(model_param)

In [229]:
# ## 事故類型依賴於肇因研判
# parent = '肇因研判子類別名稱-主要'
# child  = '事故類型及型態子類別名稱'

## 肇因研判依賴於號誌種類
parent  = '號誌-號誌種類名稱'
parent2  = '道路型態子類別名稱'
child = '肇因研判子類別名稱-主要'

In [230]:
from utils_behaviour import category_value_map

dfprob_cause = CPDs[child].sort_values('p', ascending=False)

counts = (data.groupby([child, parent, parent2]).size().reset_index(name='n'))
# counts = (data.groupby([child, parent]).size().reset_index(name='n'))

# merge cpd for p and n
dfprob_cause_counts = (
    dfprob_cause
    .merge(counts, on=[child, parent, parent2], how='left')
    # .merge(counts, on=[child, parent], how='left')
    .sort_values('p', ascending=False)
)

filtered = dfprob_cause_counts[dfprob_cause_counts['n'] >= 30].head(30)
filtered['p'] = round(filtered['p'], 4)
filtered['n'] = filtered['n'].astype(int)

# filtered[parent] = filtered[parent].map(category_value_map[parent])
# filtered[child] = filtered[child].map(category_value_map[child])
# filtered[parent2] = filtered[parent2].map(category_value_map[parent2])
filtered

肇因研判子類別名稱-主要         號誌-號誌種類名稱 道路型態子類別名稱       p      n
0          未保持行車安全距離               無號誌      高架道路  0.3929    217
1           其他不當駕車行為  行車管制號誌(附設行人專用號誌)        其他  0.3917     71
2             違反閃光號誌              閃光號誌       四岔路  0.3902   3125
4         行經圓環未依規定讓車               無號誌        圓環  0.2774    149
5          未保持行車安全距離  行車管制號誌(附設行人專用號誌)        直路  0.2658    795
7             違反閃光號誌              閃光號誌       多岔路  0.2612    216
8   無號誌路口，支線道未讓幹線道先行               無號誌       四岔路  0.2588   8563
9          未保持行車安全距離            行車管制號誌        直路  0.2576   2492
10         未保持行車安全距離               無號誌        橋樑  0.2495    350
12          其他不當駕車行為               無號誌        其他  0.2162    360
14          其他不當駕車行為               無號誌       地下道  0.2089     52
15          尚未發現肇事因素            行車管制號誌        其他  0.2044     74
17         未保持行車安全距離               無號誌       地下道  0.1928     48
18  無號誌路口，轉彎車未讓直行車先行               無號誌       三岔路  0.1830   7445
20  無號誌路口，左方車未讓右方車先行               無號誌       四岔路  0.1813   6000
22         未保持行車安全距離              閃光號誌        直路  0.1756    273
23          其他不當駕車行為               無號誌    彎曲路及附近  0.1744   1487
26         未保持行車安全距離            行車管制號誌    彎曲路及附近  0.1643     32
27  有號誌路口，轉彎車未讓直行車先行            行車管制號誌       四岔路  0.1632   7607
28          其他不當駕車行為               無號誌        坡路  0.1607    155
32         未保持行車安全距離  行車管制號誌(附設行人專用號誌)       三岔路  0.1558    938
37          其他未依規定讓車              閃光號誌       四岔路  0.1496   1198
38    恍神、緊張、心不在焉分心駕駛               無號誌        橋樑  0.1476    207
39          其他不當駕車行為            行車管制號誌        其他  0.1438     52
40          尚未發現肇事因素               無號誌        其他  0.1436    239
42          其他不當駕車行為               無號誌        橋樑  0.1404    197
45  有號誌路口，轉彎車未讓直行車先行  行車管制號誌(附設行人專用號誌)       四岔路  0.1378   2641
46    恍神、緊張、心不在焉分心駕駛               無號誌      高架道路  0.1378     76
47         未保持行車安全距離            行車管制號誌       三岔路  0.1369   2582
48         未保持行車安全距離               無號誌        直路  0.1360  13343

In [ ]:
for i in filtered['p']:
    print(i)

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from utils_behaviour import BubbleChart

filtered['特徵組合'] = filtered[parent] + '\n->' + filtered[child]
# filtered['特徵組合'] = filtered[parent] + '+\n' + filtered[parent2] + '\n->' + filtered[child]

labels = filtered['特徵組合'].reset_index(drop=True)
n_values =  filtered['n'].reset_index(drop=True)

cmap = cm.Blues
norm = mcolors.Normalize(vmin=n_values.min(), vmax=n_values.max())
colors = cmap(norm(n_values.values))

bubble_chart = BubbleChart(area=filtered['p'],
                           bubble_spacing=0.1, text_rotation=20)
bubble_chart.collapse()

fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"), figsize=(18,10))
bubble_chart.plot(ax, labels, colors)
for t in ax.texts:
    t.set_fontsize(8)

ax.axis("off")
ax.relim()
ax.autoscale_view()
ax.set_title('Bubble Chart')

sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
fig.colorbar(sm, ax=ax, label='n value')

plt.show()
